In [1]:
!pip install captum
%pip install 'drive/MyDrive/encoder_attribution_priors/.'

Processing ./drive/MyDrive/encoder_attribution_priors
  Preparing metadata (setup.py) ... done
  Created wheel for lfxai: filename=lfxai-0.1.1-py3-none-any.whl size=30914 sha256=d5b58aebe77fc772086f5766b705fb039d58838d77c4bbfe212ad87a65ee93d5
  Stored in directory: /tmp/pip-ephem-wheel-cache-q_f0qee5/wheels/7b/00/41/6fc986ab56840bbe22a1da3b92c41c9d55e2dcd851163270b7
Successfully built lfxai
  Attempting uninstall: lfxai
    Found existing installation: lfxai 0.1.1
    Uninstalling lfxai-0.1.1:
      Successfully uninstalled lfxai-0.1.1


In [2]:
import argparse
import csv
import itertools
import logging
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torchvision
from captum.attr import GradientShap, IntegratedGradients, Saliency
from scipy.stats import spearmanr
from torch.utils.data import DataLoader, RandomSampler, Subset
from torchvision import transforms

from lfxai.explanations.examples import (
    InfluenceFunctions,
    NearestNeighbours,
    SimplEx,
    TracIn,
)
from lfxai.explanations.features import attribute_auxiliary, attribute_individual_dim, tensor_attribution, attribute_training
from lfxai.models.images import (
    VAE,
    AutoEncoderMnist,
    ClassifierMnist,
    DecoderBurgess,
    DecoderMnist,
    EncoderBurgess,
    EncoderMnist,
)
from lfxai.models.losses import BetaHLoss, BtcvaeLoss, EntropyLoss, PearsonLoss, TotalVariationLoss
from lfxai.models.pretext import Identity, Mask, RandomNoise
from lfxai.utils.datasets import MaskedMNIST
from lfxai.utils.feature_attribution import generate_masks
from lfxai.utils.metrics import (
    compute_metrics,
    cos_saliency,
    count_activated_neurons,
    entropy_saliency_tensor,
    entropy_saliency,
    pearson_saliency,
    similarity_rates,
    spearman_saliency,
    pearson_saliency_tensor
)
from lfxai.utils.visualize import (
    correlation_latex_table,
    plot_pretext_saliencies,
    plot_pretext_top_example,
    plot_vae_saliencies,
    vae_box_plots,
)

In [ ]:
W=32
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
data_dir = Path.cwd() / "data/mnist"

img_size = (1, W, W)
encoder = EncoderBurgess(img_size, 3)

baseline_image = torch.zeros((1, 1, W, W), device=device)
test_dataset = torchvision.datasets.MNIST(data_dir, train=False, download=True)


test_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
test_dataset.transform = test_transform
test_dataset.data, test_dataset.targets = test_dataset.data[[1,2,3,4,5]], test_dataset.targets[[1,2,3,4,5]]
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False)

gradshap = GradientShap(encoder.mu)

In [ ]:
saliency_tensor = tensor_attribution(encoder.mu, 3, test_loader, device, gradshap, baseline_image)
saliency_array = attribute_individual_dim(encoder.mu, 3, test_loader, device, gradshap, baseline_image)

In [6]:
def disvae_feature_importance(
    random_seed: int = 1,
    batch_size: int = 300,
    n_plots: int = 20,
    n_runs: int = 5,
    dim_latent: int = 3,
    n_epochs: int = 50,
    beta_list: list = [1],
) -> None:
    # Initialize seed and device
    np.random.seed(random_seed)
    torch.random.manual_seed(random_seed)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # Load MNIST
    W = 32
    img_size = (1, W, W)
    data_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/data/mnist"
    train_dataset = torchvision.datasets.MNIST(data_dir, train=True, download=True)
    test_dataset = torchvision.datasets.MNIST(data_dir, train=False, download=True)
    train_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    train_dataset.transform = train_transform
    test_dataset.transform = test_transform
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False
    )

    # Create saving directory
    save_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/vae"
    if not save_dir.exists():

        print(f"Creating saving directory {save_dir}")
        os.makedirs(save_dir)

    # Define the computed metrics and create a csv file with appropriate headers
    loss_list = [BetaHLoss()]
    metric_list = [
        pearson_saliency,
        entropy_saliency,
        count_activated_neurons,
    ]
    metric_names = [
        "Pearson Correlation",
        "Entropy",
        "Active Neurons",
    ]
    headers = ["Loss Type", "Beta"] + metric_names
    csv_path = save_dir / "metrics.csv"
    if not csv_path.is_file():
        print(f"Creating metrics csv in {csv_path}")

        with open(csv_path, "w") as csv_file:
            dw = csv.DictWriter(csv_file, delimiter=",", fieldnames=headers)
            dw.writeheader()

    for beta, loss, run in itertools.product(
        beta_list, loss_list, range(1, n_runs + 1)
    ):
        # Initialize vaes
        encoder = EncoderBurgess(img_size, dim_latent)
        decoder = DecoderBurgess(img_size, dim_latent)
        loss.beta = beta
        name = f"{str(loss)}-vae_beta{beta}_run{run}"
        model = VAE(img_size, encoder, decoder, dim_latent, loss, name=name)
        print(f"Now fitting {name}")

        model.fit(device, train_loader, test_loader, save_dir, n_epochs)
        model.load_state_dict(torch.load(save_dir / (name + ".pt")), strict=False)

        # Compute test-set saliency and associated metrics
        baseline_image = torch.zeros((1, 1, W, W), device=device)
        gradshap = GradientShap(encoder.mu)
        attributions = attribute_individual_dim(
            encoder.mu, dim_latent, test_loader, device, gradshap, baseline_image
        )
        metrics = compute_metrics(attributions, metric_list)
        results_str = "\t".join(
            [f"{metric_names[k]} {metrics[k]:.2g}" for k in range(len(metric_list))]
        )
        print(f"Model {name} \t {results_str}")


        # Save the metrics
        with open(csv_path, "a", newline="") as csv_file:
            writer = csv.writer(csv_file, delimiter=",")
            writer.writerow([str(loss), beta] + metrics)

        # Plot a couple of examples
        plot_idx = [
            torch.nonzero(test_dataset.targets == (n % 10))[n // 10].item()
            for n in range(n_plots)
        ]
        images_to_plot = [test_dataset[i][0].numpy().reshape(W, W) for i in plot_idx]
        fig = plot_vae_saliencies(images_to_plot, attributions[plot_idx])
        fig.savefig(save_dir / f"{name}.pdf")
        plt.close(fig)

In [ ]:
disvae_feature_importance()

Creating saving directory /content/drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/vae
Creating metrics csv in /content/drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/vae/metrics.csv
Now fitting Beta-vae_beta1_run1


Epoch 1/50 	 
Train loss 303 	 Test loss 256 	 


Epoch 2/50 	 
Train loss 251 	 Test loss 247 	 


Epoch 3/50 	 
Train loss 245 	 Test loss 243 	 


Epoch 4/50 	 
Train loss 232 	 Test loss 224 	 


...

Epoch 31/50 	 
Train loss 179 	 Test loss 180 	 


Epoch 32/50 	 
Train loss 179 	 Test loss 180 	 


Epoch 33/50 	 
Train loss 179 	 Test loss 180 	 


Epoch 34/50 	 
Train loss 178 	 Test loss 179 	 


 16%|█▌        | 32/200 [00:02<00:13, 12.87batches/s]

In [ ]:
 def entropy_vae(
    random_seed: int = 1,
    batch_size: int = 300,
    n_plots: int = 20,
    n_runs: int = 1,
    dim_latent: int = 3,
    n_epochs: int = 50,
    alpha_list: list = [0.5],
) -> None:
    # Initialize seed and device
    np.random.seed(random_seed)
    torch.random.manual_seed(random_seed)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # Load MNIST
    W = 32
    img_size = (1, W, W)
    data_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/data/mnist"
    train_dataset = torchvision.datasets.MNIST(data_dir, train=True, download=True)
    #train_dataset.data, train_dataset.targets = train_dataset.data[[i for i in range(50)]], train_dataset.targets[[i for i in range(50)]]
    test_dataset = torchvision.datasets.MNIST(data_dir, train=False, download=True)
    #test_dataset.data, test_dataset.targets = test_dataset.data[[i for i in range(50)]], test_dataset.targets[[i for i in range(50)]]
    train_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    train_dataset.transform = train_transform
    test_dataset.transform = test_transform
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False
    )

    # Create saving directory
    save_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/entropy_vae"
    if not save_dir.exists():

        print(f"Creating saving directory {save_dir}")
        os.makedirs(save_dir)

    # Define the computed metrics and create a csv file with appropriate headers
    loss_list = [EntropyLoss()]
    metric_list = [
        pearson_saliency,
        entropy_saliency,
        count_activated_neurons,
    ]
    metric_names = [
        "Pearson Correlation",
        "Entropy",
        "Active Neurons",
    ]
    headers = ["Loss Type", "Alpha"] + metric_names
    csv_path = save_dir / "metrics.csv"
    if not csv_path.is_file():
        print(f"Creating metrics csv in {csv_path}")

        with open(csv_path, "w") as csv_file:
            dw = csv.DictWriter(csv_file, delimiter=",", fieldnames=headers)
            dw.writeheader()

    for alpha, loss, run in itertools.product(
        alpha_list, loss_list, range(1, n_runs + 1)
    ):
        # Initialize vaes
        encoder = EncoderBurgess(img_size, dim_latent)
        decoder = DecoderBurgess(img_size, dim_latent)
        loss.alpha = alpha
        name = f"{str(loss)}-vae_alpha{alpha}_run{run}"
        model = VAE(img_size, encoder, decoder, dim_latent, loss, name=name)
        print(f"Now fitting {name}")

        model.fit(device, train_loader, test_loader, save_dir, n_epochs)
        model.load_state_dict(torch.load(save_dir / (name + ".pt")), strict=False)

        # Compute test-set saliency and associated metrics
        baseline_image = torch.zeros((1, 1, W, W), device=device)
        gradshap = GradientShap(encoder.mu)
        attributions = attribute_individual_dim(
            encoder.mu, dim_latent, test_loader, device, gradshap, baseline_image
        )
        metrics = compute_metrics(attributions, metric_list)
        results_str = "\t".join(
            [f"{metric_names[k]} {metrics[k]:.2g}" for k in range(len(metric_list))]
        )
        print(f"Model {name} \t {results_str}")


        # Save the metrics
        with open(csv_path, "a", newline="") as csv_file:
            writer = csv.writer(csv_file, delimiter=",")
            writer.writerow([str(loss), alpha] + metrics)

        # Plot a couple of examples
        plot_idx = [
            torch.nonzero(test_dataset.targets == (n % 10))[n // 10].item()
            for n in range(n_plots)
        ]
        images_to_plot = [test_dataset[i][0].numpy().reshape(W, W) for i in plot_idx]
        fig = plot_vae_saliencies(images_to_plot, attributions[plot_idx])
        fig.savefig(save_dir / f"{name}.pdf")
        plt.close(fig)

In [ ]:
entropy_vae()

Now fitting Entropy-vae_alpha0.5_run1


Epoch 1/50 	 
Train loss 305 	 Test loss 285 	 


Epoch 2/50 	 
Train loss 256 	 Test loss 252 	 


Epoch 3/50 	 
Train loss 249 	 Test loss 247 	 


Epoch 4/50 	 
Train loss 240 	 Test loss 248 	 


...

Epoch 25/50 	 
Train loss 194 	 Test loss 204 	 


Epoch 26/50 	 
Train loss 194 	 Test loss 202 	 


Epoch 27/50 	 
Train loss 193 	 Test loss 204 	 


Epoch 28/50 	 
Train loss 192 	 Test loss 203 	 


Epoch 29/50 	 
Train loss 192 	 Test loss 206 	 
Model Entropy-vae_alpha0.5_run1 	 Pearson Correlation 0.27	Entropy 0.4	Active Neurons 1.2


In [9]:
def pearson_vae(
    random_seed: int = 1,
    batch_size: int = 300,
    n_plots: int = 20,
    n_runs: int = 1,
    dim_latent: int = 3,
    n_epochs: int = 50,
    alpha_list: list = [5],
) -> None:
    # Initialize seed and device
    np.random.seed(random_seed)
    torch.random.manual_seed(random_seed)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # Load MNIST
    W = 32
    img_size = (1, W, W)
    data_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/data/mnist"
    train_dataset = torchvision.datasets.MNIST(data_dir, train=True, download=True)
    #train_dataset.data, train_dataset.targets = train_dataset.data[[i for i in range(50)]], train_dataset.targets[[i for i in range(50)]]
    test_dataset = torchvision.datasets.MNIST(data_dir, train=False, download=True)
    #test_dataset.data, test_dataset.targets = test_dataset.data[[i for i in range(50)]], test_dataset.targets[[i for i in range(50)]]
    train_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    train_dataset.transform = train_transform
    test_dataset.transform = test_transform
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False
    )

    # Create saving directory
    save_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/pearson_vae"
    if not save_dir.exists():

        print(f"Creating saving directory {save_dir}")
        os.makedirs(save_dir)

    # Define the computed metrics and create a csv file with appropriate headers
    loss_list = [PearsonLoss()]
    metric_list = [
        pearson_saliency,
        entropy_saliency,
        count_activated_neurons,
    ]
    metric_names = [
        "Pearson Correlation",
        "Entropy",
        "Active Neurons",
    ]
    headers = ["Loss Type", "Alpha"] + metric_names
    csv_path = save_dir / "metrics.csv"
    if not csv_path.is_file():
        print(f"Creating metrics csv in {csv_path}")

        with open(csv_path, "w") as csv_file:
            dw = csv.DictWriter(csv_file, delimiter=",", fieldnames=headers)
            dw.writeheader()

    for alpha, loss, run in itertools.product(
        alpha_list, loss_list, range(1, n_runs + 1)
    ):
        # Initialize vaes
        encoder = EncoderBurgess(img_size, dim_latent)
        decoder = DecoderBurgess(img_size, dim_latent)
        loss.alpha = alpha
        name = f"{str(loss)}-vae_alpha{alpha}_run{run}"
        model = VAE(img_size, encoder, decoder, dim_latent, loss, name=name)
        print(f"Now fitting {name}")

        model.fit(device, train_loader, test_loader, save_dir, n_epochs)
        model.load_state_dict(torch.load(save_dir / (name + ".pt")), strict=False)

        # Compute test-set saliency and associated metrics
        baseline_image = torch.zeros((1, 1, W, W), device=device)
        gradshap = GradientShap(encoder.mu)
        attributions = attribute_individual_dim(
            encoder.mu, dim_latent, test_loader, device, gradshap, baseline_image
        )
        metrics = compute_metrics(attributions, metric_list)
        results_str = "\t".join(
            [f"{metric_names[k]} {metrics[k]:.2g}" for k in range(len(metric_list))]
        )
        print(f"Model {name} \t {results_str}")


        # Save the metrics
        with open(csv_path, "a", newline="") as csv_file:
            writer = csv.writer(csv_file, delimiter=",")
            writer.writerow([str(loss), alpha] + metrics)

        # Plot a couple of examples
        plot_idx = [
            torch.nonzero(test_dataset.targets == (n % 10))[n // 10].item()
            for n in range(n_plots)
        ]
        images_to_plot = [test_dataset[i][0].numpy().reshape(W, W) for i in plot_idx]
        fig = plot_vae_saliencies(images_to_plot, attributions[plot_idx])
        fig.savefig(save_dir / f"{name}.pdf")
        plt.close(fig)

In [ ]:
pearson_vae()

Now fitting Pearson-vae_alpha5_run1


Epoch 1/50 	 
Train loss 302 	 Test loss 254 	 


Epoch 2/50 	 
Train loss 235 	 Test loss 214 	 


Epoch 3/50 	 
Train loss 206 	 Test loss 200 	 


Epoch 4/50 	 
Train loss 198 	 Test loss 195 	 


...

Epoch 46/50 	 
Train loss 176 	 Test loss 178 	 


Epoch 47/50 	 
Train loss 176 	 Test loss 178 	 


Epoch 48/50 	 
Train loss 176 	 Test loss 178 	 


Epoch 49/50 	 
Train loss 175 	 Test loss 178 	 


Epoch 50/50 	 
Train loss 175 	 Test loss 177 	 
Model Pearson-vae_alpha5_run1 	 Pearson Correlation 0.14	Entropy 0.66	Active Neurons 1.2


In [10]:
pearson_vae(alpha_list = [10])

Now fitting Pearson-vae_alpha10_run1


Epoch 1/50 	 
Train loss 303 	 Test loss 255 	 


Epoch 2/50 	 
Train loss 238 	 Test loss 231 	 


Epoch 3/50 	 
Train loss 224 	 Test loss 221 	 


Epoch 4/50 	 
Train loss 214 	 Test loss 207 	 


...

Epoch 46/50 	 
Train loss 175 	 Test loss 177 	 


Epoch 47/50 	 
Train loss 175 	 Test loss 177 	 


Epoch 48/50 	 
Train loss 175 	 Test loss 176 	 


Epoch 49/50 	 
Train loss 175 	 Test loss 176 	 


Epoch 50/50 	 
Train loss 174 	 Test loss 177 	 
Model Pearson-vae_alpha10_run1 	 Pearson Correlation 0.084	Entropy 0.63	Active Neurons 1.2


In [11]:
pearson_vae(alpha_list = [50])

Now fitting Pearson-vae_alpha50_run1


Epoch 1/50 	 
Train loss 306 	 Test loss 252 	 


Epoch 2/50 	 
Train loss 239 	 Test loss 231 	 


Epoch 3/50 	 
Train loss 221 	 Test loss 214 	 


Epoch 4/50 	 
Train loss 207 	 Test loss 205 	 


...

Epoch 46/50 	 
Train loss 179 	 Test loss 180 	 


Epoch 47/50 	 
Train loss 179 	 Test loss 180 	 


Epoch 48/50 	 
Train loss 178 	 Test loss 181 	 


Epoch 49/50 	 
Train loss 178 	 Test loss 181 	 


Epoch 50/50 	 
Train loss 178 	 Test loss 180 	 
Model Pearson-vae_alpha50_run1 	 Pearson Correlation 0.014	Entropy 0.62	Active Neurons 1.2


In [3]:
def total_variation_vae(
    random_seed: int = 1,
    batch_size: int = 300,
    n_plots: int = 20,
    n_runs: int = 1,
    dim_latent: int = 3,
    n_epochs: int = 3,
    alpha_list: list = [5],
) -> None:
    # Initialize seed and device
    np.random.seed(random_seed)
    torch.random.manual_seed(random_seed)
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # Load MNIST
    W = 32
    img_size = (1, W, W)
    data_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/data/mnist"
    train_dataset = torchvision.datasets.MNIST(data_dir, train=True, download=True)
    #train_dataset.data, train_dataset.targets = train_dataset.data[[i for i in range(50)]], train_dataset.targets[[i for i in range(50)]]
    test_dataset = torchvision.datasets.MNIST(data_dir, train=False, download=True)
    #test_dataset.data, test_dataset.targets = test_dataset.data[[i for i in range(50)]], test_dataset.targets[[i for i in range(50)]]
    train_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    test_transform = transforms.Compose([transforms.Resize(W), transforms.ToTensor()])
    train_dataset.transform = train_transform
    test_dataset.transform = test_transform
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False
    )

    # Create saving directory
    save_dir = Path.cwd() / "drive/MyDrive/encoder_attribution_priors/experiments/results/mnist/total_variation_vae"
    if not save_dir.exists():

        print(f"Creating saving directory {save_dir}")
        os.makedirs(save_dir)

    # Define the computed metrics and create a csv file with appropriate headers
    loss_list = [TotalVariationLoss()]
    metric_list = [
        pearson_saliency,
        entropy_saliency,
        count_activated_neurons,
    ]
    metric_names = [
        "Pearson Correlation",
        "Entropy",
        "Active Neurons",
    ]
    headers = ["Loss Type", "Alpha"] + metric_names
    csv_path = save_dir / "metrics.csv"
    if not csv_path.is_file():
        print(f"Creating metrics csv in {csv_path}")

        with open(csv_path, "w") as csv_file:
            dw = csv.DictWriter(csv_file, delimiter=",", fieldnames=headers)
            dw.writeheader()

    for alpha, loss, run in itertools.product(
        alpha_list, loss_list, range(1, n_runs + 1)
    ):
        # Initialize vaes
        encoder = EncoderBurgess(img_size, dim_latent)
        decoder = DecoderBurgess(img_size, dim_latent)
        loss.alpha = alpha
        name = f"{str(loss)}-vae_alpha{alpha}_run{run}"
        model = VAE(img_size, encoder, decoder, dim_latent, loss, name=name)
        print(f"Now fitting {name}")

        model.fit(device, train_loader, test_loader, save_dir, n_epochs)
        model.load_state_dict(torch.load(save_dir / (name + ".pt")), strict=False)

        # Compute test-set saliency and associated metrics
        baseline_image = torch.zeros((1, 1, W, W), device=device)
        gradshap = GradientShap(encoder.mu)
        attributions = attribute_individual_dim(
            encoder.mu, dim_latent, test_loader, device, gradshap, baseline_image
        )
        metrics = compute_metrics(attributions, metric_list)
        results_str = "\t".join(
            [f"{metric_names[k]} {metrics[k]:.2g}" for k in range(len(metric_list))]
        )
        print(f"Model {name} \t {results_str}")


        # Save the metrics
        with open(csv_path, "a", newline="") as csv_file:
            writer = csv.writer(csv_file, delimiter=",")
            writer.writerow([str(loss), alpha] + metrics)

        # Plot a couple of examples
        plot_idx = [
            torch.nonzero(test_dataset.targets == (n % 10))[n // 10].item()
            for n in range(n_plots)
        ]
        images_to_plot = [test_dataset[i][0].numpy().reshape(W, W) for i in plot_idx]
        fig = plot_vae_saliencies(images_to_plot, attributions[plot_idx])
        fig.savefig(save_dir / f"{name}.pdf")
        plt.close(fig)

In [4]:
total_variation_vae()

Now fitting TotalVariation-vae_alpha5_run1


Epoch 1/3 	 
Train loss 302 	 Test loss 285 	 


Epoch 2/3 	 
Train loss 258 	 Test loss 271 	 


Epoch 3/3 	 
Train loss 255 	 Test loss 263 	 
Model TotalVariation-vae_alpha5_run1 	 Pearson Correlation 0.63	Entropy 0.73	Active Neurons 1.2
